# 1. Library

In [1]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np


In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', 500)

---

# 3. Data 

## A. Source

Description
Note: This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart? If you are interested in estimating the uncertainty distribution of the realized values of the same series, be sure to check out its companion competition

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses. In this competition, in addition to traditional forecasting methods you’re also challenged to use machine learning to improve forecast accuracy.

The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia conducts cutting-edge forecasting research and provides business forecast training. It helps companies achieve accurate predictions, estimate the levels of uncertainty, avoiding costly mistakes, and apply best forecasting practices. The MOFC is well known for its Makridakis Competitions, the first of which ran in the 1980s.

In this competition, the fifth iteration, you will use hierarchical sales data from Walmart, the world’s largest company by revenue, to forecast daily sales for the next 28 days. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

If successful, your work will continue to advance the theory and practice of forecasting. The methods used can be applied in various business areas, such as setting up appropriate inventory or service levels. Through its business support and training, the MOFC will help distribute the tools and knowledge so others can achieve more accurate and better calibrated forecasts, reduce waste and be able to appreciate uncertainty and its risk implications.

Acknowledgements
Additional thanks go to other partner organizations and prize sponsors, National Technical University of Athens (NTUA), INSEAD, Google, Uber and IIF.

## B. Files

### Sales Dataset
sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
sales_train_evaluation.csv - Includes sales [d_1 - d_1941] 
 - item_id: The id of the product.
 - dept_id: The id of the department the product belongs to.
 - cat_id: The id of the category the product belongs to.
 - store_id: The id of the store where the product is sold.
 - state_id: The State where the store is located.
 - d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.

### Sell Price
sell_prices.csv - Contains information about the price of the products sold per store and date.
 - store_id: The id of the store where the product is sold.
 - item_id: The id of the product.
 - wm_yr_wk: The id of the week.
 - sell_price: The price of the product for the given week/store

### Calender
calendar.csv - Contains information about the dates on which the products are sold.
 - date: The date in a “y-m-d” format.
 - wm_yr_wk: The id of the week the date belongs to.
 - weekday: The type of the day (Saturday, Sunday, …, Friday).
 - wday: The id of the weekday, starting from Saturday.
 - month: The month of the date.
 - year: The year of the date.
 - event_name_1: If the date includes an event, the name of this event.
 - event_type_1: If the date includes an event, the type of this event.
 - event_name_2: If the date includes a second event, the name of this event.
 - event_type_2: If the date includes a second event, the type of this event.

In [3]:
# Sales Data Training Set (1-1913) + Validation Set (1914-1941)
sales = pd.read_csv('sales_train_evaluation.csv')
sales.name = 'sales'
print("{} records in training data with {} columns".format(sales.shape[0], sales.shape[1]))

# Calendar
calendar = pd.read_csv('calendar.csv')
calendar.name = 'calendar'
print("{} records in calendar data with {} columns".format(calendar.shape[0], calendar.shape[1]))

# Pricing
prices = pd.read_csv('sell_prices.csv')
prices.name = 'prices'
print("{} records in price data with {} columns".format(prices.shape[0], prices.shape[1]))

30490 records in training data with 1947 columns
1969 records in calendar data with 14 columns
6841121 records in price data with 4 columns


---

# 3. Exploratory Data Analysis

## A. Data Structure

### i) Sales Data

In [4]:
sales.dtypes

id          object
item_id     object
dept_id     object
cat_id      object
store_id    object
             ...  
d_1937       int64
d_1938       int64
d_1939       int64
d_1940       int64
d_1941       int64
Length: 1947, dtype: object

In [5]:
# Sales

# sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
# sales_train_evaluation.csv - Includes sales [d_1 - d_1941] 
#  - item_id: The id of the product.
#  - dept_id: The id of the department the product belongs to.
#  - cat_id: The id of the category the product belongs to.
#  - store_id: The id of the store where the product is sold.
#  - state_id: The State where the store is located.
#  - d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.

sales.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


#### Results: 

There seems to be a 'id' column that contains concatinated name (item + dept + store + state + file)

In [6]:
# Looking for number of unique values in each column.

sales.nunique(axis=0)

id          30490
item_id      3049
dept_id         7
cat_id          3
store_id       10
            ...  
d_1937         56
d_1938         54
d_1939         64
d_1940         67
d_1941         76
Length: 1947, dtype: int64

#### Results: 
Sales are from
 - 3049 unique items
 - 7 unique departments
 - 3 unique categories
 - 10 unique stores
 - Rest are day sales

In [7]:
# Summarise he count, mean, standard deviation, min, and max for numeric variables in standard notation

sales.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.445490,1.781174,1.965267,1.389374,1.271794,1.216661,1.212299,1.395605,1.689669,1.782158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.656824,4.426550,4.706284,3.313292,3.126178,3.000348,2.955910,3.514318,4.089208,4.284356
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,143.000000,156.000000,187.000000,98.000000,90.000000,77.000000,84.000000,110.000000,156.000000,117.000000


#### Results: 
There does not seem to be a day without a sale. But, they can be very random and dispersed. 

**Assumption:** There are 0 values but I am going to assume that they are indicators to zero sales that day and not missing values

In [8]:
# Categorical values check

for i in sales:
    if is_string_dtype(sales[i]) == True:
        print(sales[i].unique())

# Results: 
# id: Lots of categorical values
# item_id: Lots of categorical values
# dept_id: 7 departments (2 in Hobbies, 2 in Households, 3 in Foods)
# cat_id: 3 Categories (Hobbies, Household, Foods)
# store_id: 10 stores (4 in Carlifornia, 3 in Texas, 3 in Wisconsin)
# state_id: 3 States (Carlifornia, Texas, Wisconsin)

['HOBBIES_1_001_CA_1_evaluation' 'HOBBIES_1_002_CA_1_evaluation'
 'HOBBIES_1_003_CA_1_evaluation' ... 'FOODS_3_825_WI_3_evaluation'
 'FOODS_3_826_WI_3_evaluation' 'FOODS_3_827_WI_3_evaluation']
['HOBBIES_1_001' 'HOBBIES_1_002' 'HOBBIES_1_003' ... 'FOODS_3_825'
 'FOODS_3_826' 'FOODS_3_827']
['HOBBIES_1' 'HOBBIES_2' 'HOUSEHOLD_1' 'HOUSEHOLD_2' 'FOODS_1' 'FOODS_2'
 'FOODS_3']
['HOBBIES' 'HOUSEHOLD' 'FOODS']
['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']
['CA' 'TX' 'WI']


#### Results: 
 - id: Lots of categorical values
 - item_id: Lots of categorical values
 - dept_id: 7 departments (2 in Hobbies, 2 in Households, 3 in Foods)
 - cat_id: 3 Categories (Hobbies, Household, Foods)
 - store_id: 10 stores (4 in Carlifornia, 3 in Texas, 3 in Wisconsin)
 - state_id: 3 States (Carlifornia, Texas, Wisconsin)

We can convert dept_id, cat_id, store_id and state_id into categorical data to save memory.

In [9]:
sales.memory_usage()

Index         128
id         243920
item_id    243920
dept_id    243920
cat_id     243920
            ...  
d_1937     243920
d_1938     243920
d_1939     243920
d_1940     243920
d_1941     243920
Length: 1948, dtype: int64

In [10]:
# Sales categorical columns

categorical_sales_columns = ['dept_id', 'cat_id', 'store_id', 'state_id']

for i in categorical_sales_columns:
    sales[i] = sales[i].astype('category')
    
# Reason: To reduce memory usage.

In [11]:
sales.memory_usage()

Index         128
id         243920
item_id    243920
dept_id     30846
cat_id      30622
            ...  
d_1937     243920
d_1938     243920
d_1939     243920
d_1940     243920
d_1941     243920
Length: 1948, dtype: int64

### ii) Calendar Data

In [12]:
calendar.dtypes

date            object
wm_yr_wk         int64
weekday         object
wday             int64
month            int64
year             int64
d               object
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
snap_CA          int64
snap_TX          int64
snap_WI          int64
dtype: object

In [13]:
# Calendar

# calendar.csv - Contains information about the dates on which the products are sold.
#  - date: The date in a “y-m-d” format.
#  - wm_yr_wk: The id of the week the date belongs to.
#  - weekday: The type of the day (Saturday, Sunday, …, Friday).
#  - wday: The id of the weekday, starting from Saturday.
#  - month: The month of the date.
#  - year: The year of the date.
#  - event_name_1: If the date includes an event, the name of this event.
#  - event_type_1: If the date includes an event, the type of this event.
#  - event_name_2: If the date includes a second event, the name of this event.
#  - event_type_2: If the date includes a second event, the type of this event.

calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [14]:
# Looking for number of unique values in each column.
calendar.nunique(axis=0)

date            1969
wm_yr_wk         282
weekday            7
wday               7
month             12
year               6
d               1969
event_name_1      30
event_type_1       4
event_name_2       4
event_type_2       2
snap_CA            2
snap_TX            2
snap_WI            2
dtype: int64

#### Results: 
 - date:Has 1969 unique entries
 - Years: Data is from 6 years
 - Event_name_1 has 30 unique entries
 - Event_name_2: Has 4 unique entries
        

In [15]:
# Summarise the count, mean, standard deviation, min, and max for numeric variables in standard notation

calendar.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI
count,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000
mean,11347.086338,3.997461,6.325546,2013.288471,0.330117,0.330117,0.330117
std,155.277043,2.001141,3.416864,1.580198,0.470374,0.470374,0.470374
min,11101.000000,1.000000,1.000000,2011.000000,0.000000,0.000000,0.000000
25%,11219.000000,2.000000,3.000000,2012.000000,0.000000,0.000000,0.000000
50%,11337.000000,4.000000,6.000000,2013.000000,0.000000,0.000000,0.000000
75%,11502.000000,6.000000,9.000000,2015.000000,1.000000,1.000000,1.000000
max,11621.000000,7.000000,12.000000,2016.000000,1.000000,1.000000,1.000000


In [16]:
# Categorical values check

for i in calendar:
    if is_string_dtype(calendar[i]) == True:
        print(calendar[i].unique())


['2011-01-29' '2011-01-30' '2011-01-31' ... '2016-06-17' '2016-06-18'
 '2016-06-19']
['Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday']
['d_1' 'd_2' 'd_3' ... 'd_1967' 'd_1968' 'd_1969']
[nan 'SuperBowl' 'ValentinesDay' 'PresidentsDay' 'LentStart' 'LentWeek2'
 'StPatricksDay' 'Purim End' 'OrthodoxEaster' 'Pesach End' 'Cinco De Mayo'
 "Mother's day" 'MemorialDay' 'NBAFinalsStart' 'NBAFinalsEnd'
 "Father's day" 'IndependenceDay' 'Ramadan starts' 'Eid al-Fitr'
 'LaborDay' 'ColumbusDay' 'Halloween' 'EidAlAdha' 'VeteransDay'
 'Thanksgiving' 'Christmas' 'Chanukah End' 'NewYear' 'OrthodoxChristmas'
 'MartinLutherKingDay' 'Easter']
[nan 'Sporting' 'Cultural' 'National' 'Religious']
[nan 'Easter' 'Cinco De Mayo' 'OrthodoxEaster' "Father's day"]
[nan 'Cultural' 'Religious']


#### Results: 

- id: Lots of categorical values
- item_id: Lots of categorical values
- dept_id: 7 departments (2 in Hobbies, 2 in Households, 3 in Foods)
- cat_id: 3 Categories (Hobbies, Household, Foods)
- store_id: 10 stores (4 in Carlifornia, 3 in Texas, 3 in Wisconsin)
- state_id: 3 States (Carlifornia, Texas, Wisconsin)
- date: These seem to be unique dates
- weekday: These seem to be the days of the week. (Saturday, Sunday, Monday, Tuesday, Wednesday, Thursday, Friday).
- d: These seem to be unique day identification similar to the column names of in the sales file.
- event_name_1: The names events including "NaN"
- event_type_1: The type event including "Nan"
- event_name_2: If the date includes a second event, the name of this event. Including "Nan"
- event_type_2: If the date includes a second event, the type of this event. Including "Nan"

We can convert weekday, event_name_1, event_type_1, event_name_2 and event_type_2 into categorical data.

**Assumptions:** Where there is a "NaN" I will assume there was no event and call it "NoEvent" during categorical conversion.

In [17]:
calendar.isna()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,False,False,False,False,False,False,False,True,True,True,True,False,False,False
1,False,False,False,False,False,False,False,True,True,True,True,False,False,False
2,False,False,False,False,False,False,False,True,True,True,True,False,False,False
3,False,False,False,False,False,False,False,True,True,True,True,False,False,False
4,False,False,False,False,False,False,False,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,False,False,False,False,False,False,False,True,True,True,True,False,False,False
1965,False,False,False,False,False,False,False,True,True,True,True,False,False,False
1966,False,False,False,False,False,False,False,True,True,True,True,False,False,False
1967,False,False,False,False,False,False,False,True,True,True,True,False,False,False


In [18]:
## Calendar categorical columns

categorical_calendar_columns = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

for i in categorical_calendar_columns:
    calendar[i] = calendar[i].astype('category')

# Reason: To reduce memory usage.

In [19]:
## Impute null value with new category

# Downcast in order to save memory
# ----------------------------------------------------------------------------
# Author:  Ganesh Dhasade
# License: BSD
# Publication: https://medium.com/analytics-vidhya/ways-to-handle-categorical-column-missing-data-its-implementations-15dc4a56893
# ----------------------------------------------------------------------------
def impute_nan_create_category(df,column_name):
    """ Function to replace null value with new category.
    
    Requirements
    ------------
    Pandas
    Numpy
    
    Parameters
    ----------
    df : Pandas Dataframe
    column_name : The column name of the categorical values to be changed.
        
    Returns
    -------
    Dataframe
        
    """
    df[column_name] = np.where(df[column_name].isnull(),"NoEvent",df[column_name])
        

for i in categorical_calendar_columns:
    impute_nan_create_category(calendar,i)

calendar.head(10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NoEvent,NoEvent,NoEvent,NoEvent,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NoEvent,NoEvent,NoEvent,NoEvent,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NoEvent,NoEvent,NoEvent,NoEvent,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NoEvent,NoEvent,NoEvent,NoEvent,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NoEvent,NoEvent,NoEvent,NoEvent,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NoEvent,NoEvent,NoEvent,NoEvent,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NoEvent,NoEvent,NoEvent,NoEvent,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NoEvent,NoEvent,NoEvent,NoEvent,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NoEvent,NoEvent,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NoEvent,NoEvent,NoEvent,NoEvent,1,1,0


In [20]:
# Convert date object to date time for easier time series ananlysis.

calendar['date'] = pd.to_datetime(calendar['date'])
calendar.dtypes

date            datetime64[ns]
wm_yr_wk                 int64
weekday                 object
wday                     int64
month                    int64
year                     int64
d                       object
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
snap_CA                  int64
snap_TX                  int64
snap_WI                  int64
dtype: object

### iii) Prices Data

In [21]:
prices.dtypes

store_id       object
item_id        object
wm_yr_wk        int64
sell_price    float64
dtype: object

In [22]:
# Prices
# sell_prices.csv - Contains information about the price of the products sold per store and date.
#  - store_id: The id of the store where the product is sold.
#  - item_id: The id of the product.
#  - wm_yr_wk: The id of the week.
#  - sell_price: The price of the product for the given week/store

prices.head(5)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [23]:
# Looking for number of unique values in each column.
prices.nunique(axis=0)

store_id        10
item_id       3049
wm_yr_wk       282
sell_price    1048
dtype: int64

#### Results

- store_id: There are 10 unique stores which we have seen in sales.
- item_id: There are 3049 unique items.
- wm_yr_wk: There are 282 unique identified weeks.
- sell_price: There are 1048 unique selling price.

In [24]:
# Summarise the count, mean, standard deviation, min, and max for numeric variables in standard notation

prices.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,wm_yr_wk,sell_price
count,6841121.000000,6841121.000000
mean,11382.943423,4.410952
std,148.610026,3.408814
min,11101.000000,0.010000
25%,11247.000000,2.180000
50%,11411.000000,3.470000
75%,11517.000000,5.840000
max,11621.000000,107.320000


#### Results

Selling has:
 - Mean: 4.4
 - Standard Deviation: 3.4
 - lowest selling price: 0.01
 - 25% of the prices are below 2.18
 - 50% of the prices are below 3.47
 - 75% of the prices are below 5.84
 - Highest price is 107.32
 - As per the box plot there are outliers skewing prices towards high end.

In [25]:
# place holder for sales column names
sales_cols_names = [i for i in sales.columns if 'd_' in i]


## Conclusion: Exploratory Analysis - Data Structure

Sales, Calender and Prices share certain similarities in grouping for analysis:
 - Sales and Calender share
     - Units sold per 
 - Sales and Price share:
     - item_id 
     - store_id
 - Price and Calender
     - wm_yr_wk
     

These are the best use to merge the dataframes together for any sort of analysis.